In [3]:
import pandas as pd

csv_file = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine_raw = pd.read_csv(csv_file, sep=";")
wine_data = wine_raw
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [71]:
import random
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import neighbors
from sklearn import datasets
from sklearn import metrics

# Definir seed utilizada
random.seed(1001001)

# Definir as features
all_features = ['fixed acidity', 'volatile acidity', 'citric acid', 
            'residual sugar', 'chlorides', 'free sulfur dioxide', 
            'total sulfur dioxide', 'density', 'pH', 
            'sulphates', 'alcohol', 'quality']
features = ['fixed acidity', 'volatile acidity', 'citric acid', 
            'residual sugar', 'chlorides', 'free sulfur dioxide', 
            'total sulfur dioxide', 'density', 'pH', 
            'sulphates', 'alcohol']
outcome_column = 'quality'

outcome_labels = sorted(list(set(wine_data.quality)))
print(outcome_labels)

X = np.array(wine_data[features])
Y = np.array(wine_data[outcome_column])

# Separar os conjuntos de treino e teste
Wine_Train, Wine_Test = train_test_split(wine_data, test_size=0.2, stratify=wine_data[outcome_column])

# Converter os conjuntos de treino para array
X_train = np.array(Wine_Train[features])
Y_train = np.array(Wine_Train[outcome_column])
print(X_train.shape[0], Y_train.shape[0])

# Converter os conjuntos de teste para array
X_test = np.array(Wine_Test[features])
Y_test = np.array(Wine_Test[outcome_column])
print(X_test.shape[0], Y_test.shape[0])

[3, 4, 5, 6, 7, 8]
1279 1279
320 320


In [72]:
#Create an instance of K-nearest neighbor classifier
knn_model = neighbors.KNeighborsClassifier(n_neighbors=5)

#Train the classifier
knn_model.fit(X_train,Y_train)

#Compute the prediction according to the model
Yhat = knn_model.predict(X_test)

print ('Predicted value: ' + str(Yhat[-1]), ', real target: ' + str(Y[-1]))

accuracy_train = knn_model.score(X_train, Y_train)
print('accuracy in training data:', '{:6.4f}'.format(accuracy_train))
accuracy_test = knn_model.score(X_test, Y_test)
print('accuracy in test data:    ', '{:6.4f}'.format(accuracy_test))

Predicted value: 5 , real target: 6
accuracy in training data: 0.6661
accuracy in test data:     0.4938


In [73]:
# Matriz de Confusão
confm = metrics.confusion_matrix(Y_test, Yhat, labels=outcome_labels)
confmT = confm.T

wine_ConfusionMatrix = pd.DataFrame(confmT)
wine_ConfusionMatrix.columns = ['true_' + str(val) for val in outcome_labels]
wine_ConfusionMatrix.index   = ['pred_' + str(val) for val in outcome_labels]
wine_ConfusionMatrix

,true_3,true_4,true_5,true_6,true_7,true_8
pred_3,0,0,0,0,0,0
pred_4,0,0,6,0,0,0
pred_5,0,4,88,58,4,0
pred_6,2,6,40,62,28,1
pred_7,0,1,2,8,8,2
pred_8,0,0,0,0,0,0


In [74]:
# Matriz de Confusão Normalizada
# sumPerActualQuality = wine_ConfusionMatrix.apply(sum, axis=0)
# print(sumPerActualQuality)
# dfWineCMNormalizedByActualValue = wine_ConfusionMatrix.apply((lambda x: x/sumPerActualQuality), axis=1)
# dfWineCMNormalizedByActualValue

In [99]:
# Definindo TP, TN, FP e FN para a matriz de confusão
# FIXME: Não sei se para a avaliação usamos a matriz normal gerada ou a normalizada, por enquanto estou usando a normal

TP = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    # Marcar como "true positive" os elementos que foram preditos como i e eram de fato i
    TP[i] = wine_ConfusionMatrix.loc[predicted, real]
    
FP = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    FP[i] = 0
    j = 0
    # Para os elementos que foram preditos como i
    for elements_value in wine_ConfusionMatrix.loc[predicted]:
        elements = 'true_' + str(outcome_labels[j])
        # Mas não forem de fato i, marcar como "false positive"
        if(elements != real):
            FP[i] += elements_value
        j += 1
        
FN = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    FN[i] = 0
    j = 0
    # Para os elementos que eram de fato i
    for elements_value in wine_ConfusionMatrix.loc[:,real]:
        elements = 'pred_' + str(outcome_labels[j])
        # Mas não foram preditos como i, marcar como "false negative"
        if(elements != predicted):
            FN[i] += elements_value
        j += 1

TN = {}
for i in outcome_labels:
    predicted = 'pred_' + str(i)
    real = 'true_' + str(i)
    TN[i] = 0
    # Para todos os elementos
    for index, row in wine_ConfusionMatrix.iterrows():
        # Que não foram preditos como i
        if(index != predicted):
            for j in outcome_labels:
                elements = 'true_' + str(j)
                # E que de fato não eram i, marcar como "true negative"
                if(elements != real):
                    elements_value = row[elements]
                    TN[i] += elements_value

metrics = {
    'TP': TP,
    'FP': FP,
    'FN': FN,
    'TN': TN
}

metrics

{'FN': {3: 2, 4: 11, 5: 48, 6: 66, 7: 32, 8: 3},
 'FP': {3: 0, 4: 6, 5: 66, 6: 77, 7: 13, 8: 0},
 'TN': {3: 318, 4: 303, 5: 118, 6: 115, 7: 267, 8: 317},
 'TP': {3: 0, 4: 0, 5: 88, 6: 62, 7: 8, 8: 0}}

In [26]:
# Métricas de Qualidade

# precision aka positive predictive value (PPV)
# = what fraction of the cases that my model got are true positive?
precision = TP / (TP + FP)
print('precision   ', '{:7.4f}'.format(precision))
print('precision   ', '{:7.4f}'.format(metrics.precision_score(Y_test, Yhat)), '(from sklearn.metrics)')

# recall aka sensitivity aka hit rate aka true positive rate (TPR) = TP / P
# = what fraction of the positive cases did my model get?
recall = TP / (TP + FN)
print('recall      ', '{:7.4f}'.format(recall))
print('recall      ', '{:7.4f}'.format(metrics.recall_score(Y_test, Yhat)), '(from sklearn.metrics)')

accuracy = (TP + TN) / (TP + TN + FP + FN)
print('accuracy    ', '{:7.4f}'.format(accuracy))
print('accuracy    ', '{:7.4f}'.format(metrics.accuracy_score(Y_test, Yhat)), '(from sklearn.metrics)')

F1_score = 2 * precision * recall / (precision + recall)
print('F1_score    ', '{:7.4f}'.format(F1_score))
print('F1_score    ', '{:7.4f}'.format(metrics.f1_score(Y_test, Yhat)), '(from sklearn.metrics)')


NameError: name 'TP' is not defined